<a href="https://colab.research.google.com/github/vdslab/t-taiki0620/blob/main/Okapi_BM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install unidic-lite
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
!pip install mojimoji

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.8)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.13)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+ma

In [62]:
import os
import json
import csv
import pandas as pd
import MeCab
import subprocess
import mojimoji

path = '/content/drive/MyDrive/卒業演習/tsne_+_clusters_list.json'

with open(path, encoding='utf8')as f:
  df = pd.read_json(f)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22115 entries, 0 to 22114
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   22115 non-null  int64  
 1   プロジェクトID             22115 non-null  int64  
 2   公開年度                 22115 non-null  int64  
 3   事業番号1                22115 non-null  object 
 4   事業番号2                22115 non-null  int64  
 5   事業番号3                150 non-null    float64
 6   主要政策・施策              22115 non-null  object 
 7   府省庁                  22115 non-null  object 
 8   事業名                  22115 non-null  object 
 9   事業の目的                22115 non-null  object 
 10  事業概要                 22115 non-null  object 
 11  執行額                  22115 non-null  float64
 12  objective_x          22115 non-null  float64
 13  objective_y          22115 non-null  float64
 14  cluster_in_ministry  22115 non-null  int64  
 15  ALL                  22115 non-null 

In [63]:
df

,ID,プロジェクトID,公開年度,事業番号1,事業番号2,事業番号3,主要政策・施策,府省庁,事業名,事業の目的,事業概要,執行額,objective_x,objective_y,cluster_in_ministry,ALL,高齢社会対策,男女共同参画,少子化社会対策,子ども・若者育成支援,国土強靱化施策,障害者施策,地方創生,科学技術・イノベーション,ＩＴ戦略,一億総活躍推進,海洋政策,観光立国,宇宙開発利用,地球温暖化対策,ＯＤＡ,自殺対策,医療分野の研究開発関連,沖縄振興,知的財産,2020年東京オリパラ,犯罪被害者等施策,クールジャパン,食育推進,その他
0,13,16,2015,nan,12,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、迅速・正確に内外に分かりやすく伝え...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,123.00,-3.427058,36.508163,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10604,16,2017,nan,12,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,120.00,-3.443014,36.496967,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,24982,16,2019,nan,21,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,106.00,-3.433842,36.497105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,15765,16,2018,nan,11,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,77.00,-3.439199,36.503059,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,5377,16,2016,nan,12,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、迅速・正確に内外に分かりやすく伝え...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,85.00,-3.429540,36.508717,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22110,22976,8983,2019,nan,117,NaN,['その他'],経済産業省,中小企業・小規模事業者人材対策事業,（１）各地の中小企業・小規模事業者が必要とする人材を地域内外から発掘し、マッチング・定着など...,（１）①地域中小企業人材確保支援等事業（中核人材確保スキーム事業を除く）：各地域の中小企業・...,1628.00,-0.105321,-19.544006,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22111,25938,44,2019,nan,150,NaN,['その他'],内閣府,中長期の経済運営に必要な経費,中長期の経済財政、社会基盤、社会保障等に係る重要政策の企画立案・検討に活用することを目的とし...,経済財政諮問会議等での中長期の経済財政政策の企画立案・検討に活用することを目的として、意見交...,30.73,28.377846,-26.652079,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22112,25500,4895,2019,nan,204,NaN,"['観光立国', '高齢社会対策', '子ども・若者育成支援', '障害者施策', '食育推...",農林水産省,農山漁村振興交付金,都市部において農山漁村の価値が再認識されるなかで、農山漁村の活性化、自立及び維持発展を図る...,① 地域活性化対策【補助率：定額】\n 農山漁村が持つ豊かな自然や「食」を活用した地域の活...,7282.00,-7.667183,-13.079095,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22113,13514,4916,2017,nan,38,NaN,['地方創生'],農林水産省,農林水産物の品目別輸出促進緊急対策事業,「総合的なＴＰＰ関連政策大綱」においては、高品質な我が国農林水産物の一層の輸出拡大により、「...,「農林水産業の輸出力強化戦略」及び「日本再興戦略2016」に基づき、コメ、青果物、花き、茶、...,431.00,-43.182682,-32.702126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [64]:
df = df.drop(columns=['高齢社会対策', '男女共同参画', '少子化社会対策', '子ども・若者育成支援', '国土強靱化施策', '障害者施策', '地方創生', '科学技術・イノベーション', 'ＩＴ戦略', '一億総活躍推進', '海洋政策', '観光立国', '宇宙開発利用', '地球温暖化対策', 'ＯＤＡ', '自殺対策', '医療分野の研究開発関連', '沖縄振興', '知的財産', '2020年東京オリパラ', '犯罪被害者等施策', 'クールジャパン', '食育推進', 'その他'])

In [65]:
df = df.drop(columns=['cluster_in_ministry', 'ALL'])

In [66]:
df_2018 = df.iloc[0:0]
print(df_2018)
cnt = 0
for i in range(len(df)):
  if df.loc[i]['公開年度'] == 2018:
    df_2018.loc[cnt] = df.loc[i]
    cnt += 1

Empty DataFrame
Columns: [ID, プロジェクトID, 公開年度, 事業番号1, 事業番号2, 事業番号3, 主要政策・施策, 府省庁, 事業名, 事業の目的, 事業概要, 執行額, objective_x, objective_y]
Index: []


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [8]:
df_2018

,ID,プロジェクトID,公開年度,事業番号1,事業番号2,事業番号3,主要政策・施策,府省庁,事業名,事業の目的,事業概要,執行額,objective_x,objective_y
0,15765,16,2018,nan,11,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,77.0,-3.439199,36.503059
1,15755,15,2018,nan,1,NaN,['その他'],内閣官房,内閣人事局経費,政府全体を通じた成果向上及び人材育成を狙いとして実施する。\n各研修に応じ、①幹部候補育成課...,１．官民合同セミナー（幹部級）・(課長級)（幹部級：本府省の幹部職員並びに民間企業の取締役ク...,23.0,57.182091,-2.059250
2,15926,52,2018,nan,124,NaN,['その他'],内閣府,人道救援物資備蓄経費,国際連合平和維持活動等に対する協力に関する法律（平成４年法律第７９号。以下「国際平和協力法」...,国際連合等から人道的な国際救援活動への物資協力の要請をうけた場合、迅速に対応できるよう人道救...,59.0,46.140083,-57.230614
3,15964,35,2018,nan,162,NaN,['クールジャパン'],内閣府,クールジャパン戦略推進経費,ゲーム・マンガ・アニメといったコンテンツ、ファッション、産品、日本食、伝統文化、デザイン等、...,クールジャパン担当大臣等の海外出張を活用したクールジャパン発信イベント、クールジャパン戦略の...,49.0,2.651179,-55.060982
4,15924,84,2018,nan,122,NaN,['その他'],内閣府,国際平和協力隊の派遣等経費,国際連合平和維持活動等に対する協力に関する法律（平成４年法律第７９号。以下「国際平和協力法」...,国際連合の要請に基づき、引き続き南スーダンにおいて、国際平和協力業務を実施する。,78.0,46.272320,-57.395348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4368,20146,3574,2018,nan,32,NaN,['地球温暖化対策'],環境省,短期寿命気候汚染物質削減に関する国際パートナーシップ拠出金関連業務,国内におけるブラックカーボン等の短期寿命気候汚染物質（SLCP）の排出実態及び対策技術等に係...,１．短期寿命気候汚染物質削減に関する国際パートナーシップ拠出金\n CCACに対し、平成24...,327.0,-10.464338,-70.462219
4369,15941,44,2018,nan,139,NaN,['その他'],内閣府,中長期の経済運営に必要な経費,中長期の経済財政、社会基盤、社会保障等に係る重要政策の企画立案・検討に活用することを目的とし...,経済財政諮問会議等での中長期の経済財政政策の企画立案・検討に活用することを目的として、意見交...,21.6,28.345156,-26.662083
4370,18840,4895,2018,nan,204,NaN,"['観光立国', '高齢社会対策', '子ども・若者育成支援', '障害者施策', '食育推...",農林水産省,農山漁村振興交付金,都市部において農山漁村の価値が再認識されるなかで、農山漁村の活性化、自立及び維持発展を図る...,１ 地域活性化対策【補助率：定額】\n 農山漁村が持つ豊かな自然や「食」を活用した地域の活...,7886.0,-7.659595,-13.079552
4371,19154,3708,2018,nan,119,NaN,['その他'],経済産業省,中小企業・小規模事業者人材対策事業,（１）各地の中小企業・小規模事業者が必要とする人材を地域内外から発掘し、マッチング・定着など...,（１）①地域中小企業人材確保支援等事業（中核人材確保スキーム事業を除く）：各地域の中小企業・...,1454.0,-0.068404,-19.573669


In [67]:
import re
def normalize_text(text):
  # 全部全角に変換
  result = mojimoji.han_to_zen(text)

  # 全角から半角に変換（カナは除く）
  result = mojimoji.zen_to_han(result, kana=False)

  # 全ての文字を小文字に変換
  result = result.lower()
  
  return result
  # return re.sub(' ', '', mojimoji.zen_to_han(text)).upper().split()

In [69]:
# 0404 used this
cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
def tokenize(text):
  m=MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
  lines = m.parse(text).splitlines()
  words = []
  for line in lines:
    chunks = line.split('\t')
    # print(chunks[0])
    if chunks[0] != 'EOS' and not chunks[1].startswith('記号'):
      words.append(chunks[0])
  return words

In [70]:
from gensim.models import word2vec
model   = word2vec.Word2Vec.load("/content/drive/MyDrive/Colab Notebooks/pre-judgit/ja-MeCab-50.data.model")

In [77]:
#分かち書きする文
separation = "事業の目的"
#separation = "事業概要"

In [78]:
tf_business_list = []
word_list = []
for i in range(len(df_2018)):
  tf_business_list.append(df_2018.loc[i])
  if df_2018.loc[i][separation] != '':
    if normalize_text(df_2018.loc[i][separation]) != []:
      words = tokenize(normalize_text(df_2018.loc[i][separation]))
      for word in words:
        if word not in word_list:
          word_list.append(word)

In [79]:
print(len(tf_business_list), len(word_list))
print(tf_business_list[0])
print(word_list)

4373 13771
ID                                                         15765
プロジェクトID                                                      16
公開年度                                                        2018
事業番号1                                                        nan
事業番号2                                                         11
事業番号3                                                        NaN
主要政策・施策                                                  ['その他']
府省庁                                                         内閣官房
事業名                                                       内閣広報経費
事業の目的          内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...
事業概要           内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...
執行額                                                           77
objective_x                                              -3.4392
objective_y                                              36.5031
Name: 0, dtype: object
['内閣', 'の', '重要', '施策', 'に関する', '広報', 'を', '首相官邸', 'ホームペ

In [80]:
words = tokenize(normalize_text(tf_business_list[0].loc[separation])[0])

In [81]:
tf = {}
for data in tf_business_list:
  tf[data.loc["プロジェクトID"]] = {}
  words = tokenize(normalize_text(data.loc[separation]))
  tmp = {}
  for word in words:
    tmp[word] = words.count(word)/len(words)
  tf[data.loc["プロジェクトID"]] = tmp

In [82]:
tf[4924]

{'ある': 0.012658227848101266,
 'い': 0.012658227848101266,
 'いく': 0.012658227848101266,
 'から': 0.012658227848101266,
 'が': 0.0379746835443038,
 'こと': 0.02531645569620253,
 'し': 0.02531645569620253,
 'する': 0.05063291139240506,
 'た': 0.02531645569620253,
 'ため': 0.02531645569620253,
 'て': 0.02531645569620253,
 'で': 0.02531645569620253,
 'ない': 0.012658227848101266,
 'に': 0.0379746835443038,
 'において': 0.012658227848101266,
 'により': 0.012658227848101266,
 'の': 0.0379746835443038,
 'は': 0.012658227848101266,
 'や': 0.02531645569620253,
 'れ': 0.012658227848101266,
 'を': 0.06329113924050633,
 'システム': 0.012658227848101266,
 'モデル': 0.012658227848101266,
 '不足': 0.012658227848101266,
 '中': 0.012658227848101266,
 '作業量': 0.012658227848101266,
 '力': 0.012658227848101266,
 '労働': 0.012658227848101266,
 '労働力': 0.02531645569620253,
 '化': 0.012658227848101266,
 '各': 0.012658227848101266,
 '大規模化': 0.012658227848101266,
 '就業': 0.012658227848101266,
 '平準': 0.012658227848101266,
 '年間': 0.012658227848101266,
 '必要': 

In [83]:
df_2018

,ID,プロジェクトID,公開年度,事業番号1,事業番号2,事業番号3,主要政策・施策,府省庁,事業名,事業の目的,事業概要,執行額,objective_x,objective_y
0,15765,16,2018,nan,11,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,77.0,-3.439199,36.503059
1,15755,15,2018,nan,1,NaN,['その他'],内閣官房,内閣人事局経費,政府全体を通じた成果向上及び人材育成を狙いとして実施する。\n各研修に応じ、①幹部候補育成課...,１．官民合同セミナー（幹部級）・(課長級)（幹部級：本府省の幹部職員並びに民間企業の取締役ク...,23.0,57.182091,-2.059250
2,15926,52,2018,nan,124,NaN,['その他'],内閣府,人道救援物資備蓄経費,国際連合平和維持活動等に対する協力に関する法律（平成４年法律第７９号。以下「国際平和協力法」...,国際連合等から人道的な国際救援活動への物資協力の要請をうけた場合、迅速に対応できるよう人道救...,59.0,46.140083,-57.230614
3,15964,35,2018,nan,162,NaN,['クールジャパン'],内閣府,クールジャパン戦略推進経費,ゲーム・マンガ・アニメといったコンテンツ、ファッション、産品、日本食、伝統文化、デザイン等、...,クールジャパン担当大臣等の海外出張を活用したクールジャパン発信イベント、クールジャパン戦略の...,49.0,2.651179,-55.060982
4,15924,84,2018,nan,122,NaN,['その他'],内閣府,国際平和協力隊の派遣等経費,国際連合平和維持活動等に対する協力に関する法律（平成４年法律第７９号。以下「国際平和協力法」...,国際連合の要請に基づき、引き続き南スーダンにおいて、国際平和協力業務を実施する。,78.0,46.272320,-57.395348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4368,20146,3574,2018,nan,32,NaN,['地球温暖化対策'],環境省,短期寿命気候汚染物質削減に関する国際パートナーシップ拠出金関連業務,国内におけるブラックカーボン等の短期寿命気候汚染物質（SLCP）の排出実態及び対策技術等に係...,１．短期寿命気候汚染物質削減に関する国際パートナーシップ拠出金\n CCACに対し、平成24...,327.0,-10.464338,-70.462219
4369,15941,44,2018,nan,139,NaN,['その他'],内閣府,中長期の経済運営に必要な経費,中長期の経済財政、社会基盤、社会保障等に係る重要政策の企画立案・検討に活用することを目的とし...,経済財政諮問会議等での中長期の経済財政政策の企画立案・検討に活用することを目的として、意見交...,21.6,28.345156,-26.662083
4370,18840,4895,2018,nan,204,NaN,"['観光立国', '高齢社会対策', '子ども・若者育成支援', '障害者施策', '食育推...",農林水産省,農山漁村振興交付金,都市部において農山漁村の価値が再認識されるなかで、農山漁村の活性化、自立及び維持発展を図る...,１ 地域活性化対策【補助率：定額】\n 農山漁村が持つ豊かな自然や「食」を活用した地域の活...,7886.0,-7.659595,-13.079552
4371,19154,3708,2018,nan,119,NaN,['その他'],経済産業省,中小企業・小規模事業者人材対策事業,（１）各地の中小企業・小規模事業者が必要とする人材を地域内外から発掘し、マッチング・定着など...,（１）①地域中小企業人材確保支援等事業（中核人材確保スキーム事業を除く）：各地域の中小企業・...,1454.0,-0.068404,-19.573669


In [84]:
len(word_list)

13771

In [85]:
import math
tmp = {}
for data in tf_business_list:
  words = tokenize(normalize_text(data.loc[separation]))
  check = []
  for word in words:
    if word not in check:
      check.append(word)
      if word not in tmp.keys():
        tmp[word] = 1
      else:
        tmp[word] += 1

In [86]:
tmp

{'内閣': 3,
 'の': 4333,
 '重要': 306,
 '施策': 232,
 'に関する': 750,
 '広報': 37,
 'を': 4327,
 '首相官邸': 2,
 'ホームページ': 9,
 '等': 2904,
 '通じ': 227,
 '国内外': 95,
 'に対して': 241,
 '迅速': 182,
 '正確': 34,
 'に': 3778,
 '分かり': 11,
 'やすく': 17,
 '伝える': 6,
 'こと': 2849,
 'により': 1612,
 'へ': 793,
 '理解': 180,
 '深める': 18,
 '目的': 1531,
 'と': 2695,
 'する': 3781,
 '政府': 103,
 '全体': 129,
 'た': 1799,
 '成果': 128,
 '向上': 770,
 '及び': 1631,
 '人材育成': 55,
 '狙い': 2,
 'として': 699,
 '実施': 1177,
 '各': 211,
 '研修': 108,
 '応じ': 142,
 '幹部': 11,
 '候補': 11,
 '育成': 162,
 '課程': 7,
 '対象者': 19,
 '政策': 244,
 '企画': 82,
 '立案': 131,
 '係る': 355,
 '能力': 256,
 '資質': 40,
 '国家公務員': 9,
 '職場': 25,
 'において': 709,
 '共通': 52,
 '必要': 886,
 'な': 2664,
 '業務': 244,
 '管理': 297,
 'し': 2739,
 'て': 1937,
 'いる': 1051,
 '国際連合平和維持活動等に対する協力に関する法律': 2,
 '平成4年': 3,
 '法律': 94,
 '第': 205,
 '79': 3,
 '号': 56,
 '以下': 225,
 '国際平和協力': 10,
 '法': 132,
 'いう': 159,
 '基づき': 340,
 '国際連合': 6,
 'による': 697,
 '人道的': 3,
 '国際': 305,
 '救援': 5,
 '活動': 358,
 'に対し': 207,
 '適切': 307,
 'かつ': 475,

In [87]:
idf = {}
for word in word_list:
  idf[word] = math.log2((len(tf_business_list)-tmp[word]+0.5)/(tmp[word]+0.5))

In [88]:
idf

{'内閣': 10.286227700970304,
 'の': -6.741466986401147,
 '重要': 3.7301833668691087,
 '施策': 4.154850767313832,
 'に関する': 2.271459917364053,
 '広報': 6.853496703702325,
 'を': -6.54015929314597,
 '首相官邸': 10.7719845887782,
 'ホームページ': 8.84367315844369,
 '等': -0.9829644708692443,
 '通じ': 4.187955641910318,
 '国内外': 5.485460547209998,
 'に対して': 4.096919725343903,
 '迅速': 4.52149826033134,
 '正確': 6.974788651872586,
 'に': -2.6656402095912837,
 '分かり': 8.567377459927364,
 'やすく': 7.959670810872941,
 '伝える': 9.392152269663029,
 'こと': -0.9023726423318429,
 'により': 0.7761529668730838,
 'へ': 2.1738589389677507,
 '理解': 4.538084113839216,
 '深める': 7.879169264978851,
 '目的': 0.8922149821806525,
 'と': -0.683380372533548,
 'する': -2.6740715597940747,
 '政府': 5.366702320964096,
 '全体': 5.034570612000969,
 'た': 0.5166961678727295,
 '成果': 5.046094207984236,
 '向上': 2.2255319834932448,
 '及び': 0.749292633198691,
 '人材育成': 6.281898707934484,
 '狙い': 10.7719845887782,
 'として': 2.393151989052824,
 '実施': 1.440766034003714,
 '各': 4.29872

In [89]:
sum = 0
for word in word_list:
  sum += tmp[word]

In [90]:
sum

200800

In [91]:
avgdl = sum/len(tf_business_list)

In [92]:
avgdl

45.91813400411617

In [93]:
okapi = {}
count = 0
for data in tf_business_list:
  if data.loc['プロジェクトID'] not in okapi.keys():
    okapi[data.loc['プロジェクトID']] = {}
  words = tokenize(normalize_text(data.loc[separation]))
  ex = {}
  # print(count, data.loc['プロジェクトID'], data.loc['事業名'], words)
  count += 1
  for word in words:
    ex[word] = idf[word] * ((tf[data.loc['プロジェクトID']][word]*(2.0+1)) / (tf[data.loc['プロジェクトID']][word] + 2.0 * (1-0.75+0.75*(tmp[word]/avgdl))))
  
  # print(ex)
  okapi[data.loc['プロジェクトID']] = ex

In [96]:
result_sorted = sorted(okapi[15].items(), key=lambda x: x[1], reverse=True)

In [41]:
result_sorted

[('課長', 1.7643538391974827),
 ('幹部', 1.5640606981582585),
 ('社員', 1.3415632221140406),
 ('級', 1.3144440772735302),
 ('新任', 1.010931081597097),
 ('クラス', 0.8310260350910565),
 ('取締役', 0.6840712464961617),
 ('課長補佐', 0.6840712464961617),
 ('係長', 0.6840712464961617),
 ('新規採用', 0.610332837380605),
 ('係員', 0.610332837380605),
 ('初任', 0.5567935789777544),
 ('国家公務員', 0.4218743753380411),
 ('合同', 0.3895721868827445),
 ('府', 0.332626722754631),
 ('課程', 0.26514940638444134),
 ('省', 0.23239471005518744),
 ('候補', 0.2087576734861809),
 ('中央', 0.18552624357641598),
 ('管理者', 0.18043840167385225),
 ('民間企業', 0.14317141603372338),
 ('官民', 0.12631026563176304),
 ('セミナー', 0.09375088103939952),
 ('研修', 0.07510960499513487),
 ('職員', 0.054707192045572534),
 ('並びに', 0.04347101858075759),
 ('本', 0.03365646031581136),
 ('育成', 0.03235245745936892),
 ('対象', 0.02838960928302247),
 ('4', 0.02120642511934805),
 ('管理', 0.007987733047305847),
 ('3', 0.005773328851613824),
 ('2', 0.0027037427474359108),
 ('1', 0.00242634

In [98]:
word_score = []
tmp_list = sorted(okapi[15].items(), key=lambda x: x[1], reverse=True)
for i in range(5):
    word_score.append({"word" : tmp_list[i][0],
                     "score" : tmp_list[i][1]})
print(word_score)

[{'word': '狙い', 'score': 0.8825632320938026}, {'word': '課程', 'score': 0.587474312036277}, {'word': '国家公務員', 'score': 0.5199907987137709}, {'word': '幹部', 'score': 0.4661409328412976}, {'word': '候補', 'score': 0.4661409328412976}]


In [102]:
import json
obj = []
for data in tf_business_list:
  print(data.loc['プロジェクトID'])
  tmp_array = []
  tmp_list = sorted(okapi[data.loc['プロジェクトID']].items(), key=lambda x : x[1], reverse=True)
  for i in range(5):
    try:
      tmp_array.append({"word": tmp_list[i][0],
                         "score":float(tmp_list[i][1])})
    except:
      print("skip")
  obj.append({'プロジェクトID':int(data.loc['プロジェクトID']),
              'okapi':tmp_array})

with open('/content/drive/MyDrive/Colab Notebooks/pre-judgit/okapi2018.json', 'w', encoding='utf8')as f:
  json.dump(obj, f, indent=2, ensure_ascii=False)

16
15
52
35
84
62
63
83
12
1690
161
1850
1494
14
1677
1614
1641
1857
1385
1507
4908
1509
4337
815
1212
814
1213
816
907
4473
3182
4471
1740
1562
1564
1565
1781
1786
1560
794
939
1109
1248
1770
4838
8200
179
7614
4379
1559
795
8194
173
1233
990
879
3070
2646
3094
1234
3093
4451
4462
8344
4492
1517
4450
5267
3831
4480
4491
60
4521
4487
5254
8322
8293
4046
391
390
388
389
393
392
992
997
403
404
4622
346
345
1003
342
348
409
410
411
412
6938
343
245
849
347
1499
895
1021
720
750
719
483
1192
510
889
1166
799
1174
6935
394
1185
399
226
509
1190
8063
1186
1191
1188
2120
2082
5072
5148
5253
5149
5163
5282
1502
4518
4557
661
313
2803
5
314
657
4433
2887
2966
3186
659
2800
1449
8309
8294
658
8310
2678
1104
2662
2696
3839
1686
8295
8311
2984
1229
2697
3840
255
1228
2633
8296
8312
7812
7813
440
4551
4659
1215
7857
4418
1420
2242
880
940
833
702
742
1129
870
8071
1232
871
220
7730
4531
5320
5112
5334
5141
5164
431
671
433
105
2881
2370
2879
2883
2307
2369
2258
2308
2365
2228
2827
2931
2295
2375
4

In [103]:
okapi[3550]

{'における': 0.03943625453091387,
 '中央合同庁舎第5号館': 11.028907532246103,
 '工事': 3.5491869276043877,
 '等': -0.007710373336530736}

In [104]:
len(obj)

4373

In [60]:
for data in tf_business_list:
  if data.loc["プロジェクトID"] == 469:
    print(data)

ID                                                         18069
プロジェクトID                                                     469
公開年度                                                        2018
事業番号1                                                        nan
事業番号2                                                        436
事業番号3                                                        NaN
主要政策・施策                                                  ['その他']
府省庁                                                        厚生労働省
事業名                                                 労災保険給付に必要な経費
事業の目的          　労災保険給付は、労働者の業務上又は通勤による負傷、疾病、障害、死亡等に対して、迅速かつ公正...
事業概要                                                    　別添のとおり。
執行額                                                       737992
objective_x                                              25.1803
objective_y                                              81.7496
Name: 722, dtype: object
